<a href="https://colab.research.google.com/github/Kridr/HW_PB_HSE/blob/main/HW8_PB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 4.8 MB/s 


In [2]:
from Bio import SeqIO
import urllib.request
import re

In [3]:
l=['CP000036.1'] # id генома в базе ensemble

In [4]:
from Bio import SeqIO
import urllib.request
folder="./"
records=[]
b=[]
for genome in l:
    #раньше такая ссылка была
    #genomeurl='https://www.ebi.ac.uk/ena/data/view/'+genome+'&display=text&download=txt&filename='+genome+'.txt'
    genomeurl='https://www.ebi.ac.uk/ena/browser/api/embl/'+genome+'?download=true' # ссылка на скачивание
    print(genomeurl)
    g_file= genome + '.gbk'
    handle =urllib.request.urlretrieve(genomeurl,g_file) # скачиваем
    urllib.request.urlcleanup()

https://www.ebi.ac.uk/ena/browser/api/embl/CP000036.1?download=true


In [8]:
records=[]
b=[]
#размер upstream участка
prom_size=100
import re
from Bio.SeqRecord import SeqRecord
for genome in l:
    g_file=folder + genome + '.gbk' # путь к файлу
    print(g_file)
    handle = open(g_file) # открываем
    record=SeqIO.read(handle, "embl") # считываем запись если несколько то parse
    handle.close()
    genes=[]
    proms=[]
    c=0
    for feature in record.features:
        if feature.type=="CDS":
            try:
                f="1.5.2.1 : purine biosynthesis" in feature.qualifiers['function']
            except KeyError:
                f=False
                try:
                    if re.search("purH|folD|purL|guaA|purT|purA|purM|purC|purR",feature.qualifiers['gene'][0]):
                        c+=1
                        if feature.location.strand==1:
                            start=feature.location.start-prom_size
                            end=feature.location.start
                            s=record.seq[start:end]
                        else:
                            start=feature.location.end
                            end=feature.location.end+prom_size
                            s=record.seq[start:end].reverse_complement()
                        proms.append(SeqRecord(s, id=feature.qualifiers['gene'][0]+str(c),description=str(start)+"-"+str(end)))
                except KeyError:
                    f=False
            if f:
                c+=1
                if feature.location.strand==1:
                    start=feature.location.start-prom_size
                    end=feature.location.start
                    s=record.seq[start:end]
                else:
                    start=feature.location.end
                    end=feature.location.end+prom_size
                    s=record.seq[start:end].reverse_complement()
                proms.append(SeqRecord(s, id=feature.qualifiers['gene'][0]+str(c),description=str(start)+"-"+str(end)))
    file_name=folder + record.name + '.fasta'
    print(file_name)
    SeqIO.write(proms, file_name, "fasta")

./CP000036.1.gbk
./CP000036.fasta


In [9]:
!cat CP000036.fasta

>folD1 430671-430771
TCACGCGATAAATCTGAAACGAAACCTGACAGCGCGCCCCGCTTCTGACAAAATAGGCGC
ATCCCCTTCGACCTACGTAACAGATGGAATCCTCTCTCTG
>purT2 1155871-1155971
AGTATATTTCTTTTCGGTTGAGAAATCAAAATCAACAATAAAGACACACGCAAACGTTTT
CGTTTATACTGCGCGCGGAATTAATCAGGGGATATTCGTT
>purR3 1451148-1451248
CGGCGTACCGCAACACTTTTGTTGTGCGTAAGGTGTGTAAAGGCAAACGTTTACCTTGCG
ATTTTGCAGGAGCTGAAGTTAGGGTCTGGAGTGAAATGGA
>purC4 2487662-2487762
GTAGGTCGGATAAGGCGTTCTTGCCGCATCCGACAAAACATCCGGCACACCAGACAGCAA
AAGATTTTAAAACGTTAATTCACACCCAGGAGTGATAAAG
>purM5 2514748-2514848
AAAAGGTTGTGCAAAGCAGCCTCGCAAACGTTTGCTTTCCCTGTTAGAATTGCGCCGAAT
TTTATTTTTCTACCGCAAGTAACGCGTGGGGACCCAAGCA
>guaA6 2528258-2528358
AAGAGTCCCCGAACTACCGTCTGGGCTCCTGATTCTCTTCGCCCGACTTCATGTCGGGCG
ATTTATATTATCTGTTTCACTTGCCTCGGAATAAGCGTCA
>purL7 2592620-2592720
ATTTCCACGCAAACGGTTTCGTCAGCGCATCAGATTCTTTATAATGACGCCCGTTTCCCC
CCCCCTTGGGTACACCGAAAGCTTAGAAGACGAGAGACTT
>purH8 4065934-4066034
CGGGCGAAAAATAAGGATGCCCCGTTAGGGGCGTTAGCTGAGTTTTTCGCGAAAAATTCA
GCTAACGCTCTCTGTAATAGTCAAATCCAGGGGATTTACC
>p